# Experiments of VIS with fully factorized proposals

Run these two cells always at the beginning:

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('./pyGM/')
import numpy as np
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import pickle 
import time

from categorical import *
from inference_MF import *
from experiment_MF import *
# my_ising_grid is my own implementation which simplifies its expression
from pyGM.misc import ising_grid, my_ising_grid
from pyGM.factor import *
from pyGM.graphmodel import *

In [ ]:
my_color_map={
    "VIS-R":"tab:purple",
    "VIS-Rm":"tab:red",
    "VIS-Rh":"tab:blue",
    "VIS-I":"tab:brown",
    "DEst":"tab:orange",
    "sMC":"tab:pink",
    "VIS-Rx-rel":"tab:green",
    "VIS-Rx-0.5":"tab:cyan",
    "VIS-Rx-0.25":"tab:gray",
    "VIS-Rx-0.1":"salmon",
    "VIS-Rx-0.05":"gold"
}

def my_plot_results(real_vals, estimates, variances, theo_variances, num_samples, num_measures,
                 logscale=None, to_show_algs=[], names_to_show=[], limits_without_smallest=False, 
                    color_dict=my_color_map):
    xaxis = np.arange(0,num_samples+1,num_samples//num_measures)[1:]

    fig, ax = plt.subplots(1, 2, figsize=(15, 6)) # 10,3

    colors = [color_dict[a] for a in names_to_show]
    min_val = [[],[]]
    for col, alg_name in enumerate(to_show_algs):
        point_rel_error = np.abs(estimates[alg_name] - real_vals)/real_vals
        ax[0].plot(xaxis, np.mean(point_rel_error, axis=(0, 1)), label=names_to_show[col], color=colors[col])

        min_val[0].append(np.min(np.mean(point_rel_error, axis=(0, 1))))

        ax[1].plot(xaxis, np.mean(variances[alg_name], axis=(0, 1)), label=names_to_show[col], color=colors[col])

        min_val[1].append(np.min(np.mean(variances[alg_name], axis=(0, 1))))

    #ax[0].set_title("Estimation")
    ax[0].set_xlabel("Num. samples")
    ax[0].set_ylabel("Mean relative error")
    ax[0].legend()

    #ax[1].set_title("Variance")
    ax[1].set_xlabel("Num. samples")
    ax[1].set_ylabel("Mean variance")

    if limits_without_smallest:
        min_val[1].sort()
        x1, x2, y1, y2 = ax[1].axis()
        ax[1].axis((x1, x2, min_val[1][1]/2, y2))

    if (logscale is not None):
        for ls in logscale:
            ax[ls].set_yscale('log')
            #ax[ls].set_ylabel("Log-" + ax[ls].get_ylabel())

    plt.tight_layout()
    plt.show()

    

def my_plot_times(proj_times, logscale=False, xticks=None, to_show_algs=[], names_to_show=[]):
    xaxis = np.arange(len(proj_times))+1

    fig, ax = plt.subplots(1, 1, figsize=(10, 6)) # 10,3

    colors = ["C" + s for s in list(map(str, np.arange(len(to_show_algs))))]

    for col, alg_name in enumerate(to_show_algs):
        vals = [np.mean(proj_times[i_d][alg_name]) for i_d in np.arange(len(proj_times))]
        ax.plot(xaxis, vals, label=names_to_show[col], color=colors[col])

    #ax.set_title("Time consumption")
    ax.set_xlabel("delta")
    if xticks is not None:
        ax.set_xticks(xaxis)
        ax.set_xticklabels(xticks)
    ax.set_ylabel("time")
    ax.legend()

    if logscale:
        ax.set_yscale('log')
        #ax.set_ylabel("log-" + ax.get_ylabel())

    plt.show()



## Experiment 200602a: mixture vs others

In [ ]:
n_vars = 4
cardinality = 2

deltas = np.array([2,3.5,5,10])
f_model = lambda n,d,s : GraphModel(ising_grid(n=n,d=d,sigp=s))

                
algorithms = [ SimpleMonteCarlo(random_proposal=True),
               DeterministicEstimation(random_proposal=True),
               VIS_Sp_I_cont(r=2,rate_i=0.5,minoriting_factor=0.5,step=0.5, control_var=True),
               VIS_Sp_I(r=2,step=0.5,rate_i=0.05,fixed_i=False, deterministic_i=True, ext_name="-0.05"),
               VIS_R_sp(r=2,step=0.5, ext_name="-H"),
               VIS_R(r=2,step=0.5),
               #VIS_A(alpha=2,step=0.5),
               VIS_I(),
               VIS_Ru(r=2)
             ]

seed = 17

num_repeats = np.array([100])#10,1000
num_samples = cardinality**(n_vars**2) # cardinality**(13)
num_problems = 1
num_measures = 2**7

estimates = {}
variances = {}
theo_variances = {}
real_vals = {}

for n_rep in num_repeats:
    for s in deltas: #tqdm(weights, desc='Weight'):
        print ("Delta = ",s)
        problem_generator = lambda : f_model(n_vars,cardinality,s)
        real_vals[s], estimates[s], variances[s], _, _ = scenario(problem_generator, num_problems,
                                               n_rep, num_samples, num_measures, algorithms, rnd_seed=seed)

        #plot_results(real_vals[s],estimates[s], variances[s], theo_variances[s], 
        #             num_samples, num_measures)

    ######################## WHAT DO WE WANT TO SAVE? ########################
    data = {"real_vals": real_vals, "estimates": estimates, "variances": variances, 
            "n_rep": n_rep, "num_samples": num_samples, "num_measures": num_measures,
            "n_vars": n_vars, "cardinality": cardinality, "deltas": deltas, "seed": seed
           }

    f_pckl_name = "results/exp200602a_seed_"+str(seed)+"_nrep_"+str(n_rep)+"_"+str(int(time.time()))+".pickle"
    with open(f_pckl_name, 'wb') as f_pckl:
        pickle.dump(data, f_pckl, pickle.HIGHEST_PROTOCOL)


In [ ]:
'''files = ["exp200602_seed_7_delta_3.5_nsamp_8192_nrep_100_1591105805.pickle",
         "exp200602_seed_7_delta_3.5_nsamp_8192_nrep_1000_1591134165.pickle",
         "exp200602_seed_7_delta_10.0_nsamp_8192_nrep_100_1591105822.pickle",
         "exp200602_seed_7_delta_10.0_nsamp_8192_nrep_1000_1591134225.pickle",
         "exp200602_seed_17_delta_3.5_nsamp_8192_nrep_100_1591105813.pickle",
         "exp200602_seed_17_delta_3.5_nsamp_8192_nrep_1000_1591134184.pickle",
         "exp200602_seed_17_delta_10.0_nsamp_8192_nrep_100_1591105831.pickle",
         "exp200602_seed_17_delta_10.0_nsamp_8192_nrep_1000_1591133624.pickle"]
algs =      ["VIS-Rh-I-cont","VIS-Rh-I-0.05", "VIS-Rh-H", "VIS-Rm", "VIS-I", "VIS-R", "DEst", "sMC"]
alg_names = ["VIS-Rx-rel",   "VIS-Rx-0.05",   "VIS-Rh",   "VIS-Rm", "VIS-I", "VIS-R", "DEst", "sMC"]
to_show = [0,2,3,4,5]#,6,7]
for f in files:
    with open('results_paper/'+f, 'rb') as handle:
        res = pickle.load(handle)

        for s in res["deltas"]:
            my_plot_results(res["real_vals"][s],res["estimates"][s], res["variances"][s], None, 
                            res["num_samples"], res["num_measures"],logscale=[1], limits_without_smallest=True,
                            to_show_algs=[algs[a] for a in to_show], names_to_show=[alg_names[a] for a in to_show])
'''

## Experiment: impact of the value of n_vars


In [ ]:
algorithms = [ VIS_R_sp(r=2, step=0.5, limit_nvars=1, ext_name="-1"),
               VIS_R_sp(r=2, step=0.5, limit_nvars=2, ext_name="-2"),
               VIS_R_sp(r=2, step=0.5, limit_nvars=3, ext_name="-3"),
               VIS_R_sp(r=2, step=0.5, limit_nvars=4, ext_name="-4"),
               VIS_R_sp(r=2, step=0.5, limit_nvars=5, ext_name="-5"),
               VIS_R_sp(r=2, step=0.5, limit_nvars=6, ext_name="-6"),
               VIS_R_sp(r=2, step=0.5, limit_nvars=7, ext_name="-7"),
               VIS_R_sp(r=2, step=0.5, limit_nvars=8, ext_name="-8"),
               VIS_R_sp(r=2, step=0.5, limit_nvars=9, ext_name="-9"),
               VIS_R_sp(r=2, step=0.5, limit_nvars=10, ext_name="-10"),
               VIS_R_sp(r=2, step=0.5, limit_nvars=11, ext_name="-11"),
               VIS_R_sp(r=2, step=0.5, limit_nvars=12, ext_name="-12"),
               VIS_R_sp(r=2, step=0.5, limit_nvars=13, ext_name="-13"),
               VIS_R_sp(r=2, step=0.5, limit_nvars=14, ext_name="-14")]
''',
               VIS_R_sp(r=2, step=0.5, limit_nvars=15, ext_name="-15"),
               VIS_R_sp(r=2, step=0.5, limit_nvars=16, ext_name="-16"),
               VIS_R_sp(r=2, step=0.5, limit_nvars=17, ext_name="-17"),
               VIS_R_sp(r=2, step=0.5, limit_nvars=18, ext_name="-18"),
               VIS_R_sp(r=2, step=0.5, limit_nvars=19, ext_name="-19"),
               VIS_R_sp(r=2, step=0.5, limit_nvars=20, ext_name="-20"),
               VIS_R_sp(r=2, step=0.5, limit_nvars=21, ext_name="-21"),
               VIS_R_sp(r=2, step=0.5, limit_nvars=22, ext_name="-22"),
               VIS_R_sp(r=2, step=0.5, limit_nvars=23, ext_name="-23"),
               VIS_R_sp(r=2, step=0.5, limit_nvars=24, ext_name="-24")'''
                

n_vars = 4
cardinality = 2

deltas = np.array([2, 3.5, 5, 10])
f_model = lambda n,d,s : GraphModel(ising_grid(n=n,d=d,sigp=s))
                

seed = 17

num_problems = 20
num_repeats = 100
num_samples = cardinality**13
num_measures = cardinality**7

estimates = {}
variances = {}
real_vals = {}
times = {}

for s in deltas: #tqdm(weights, desc='Weight'):
    print ("Delta = ",s)
    problem_generator = lambda : f_model(n_vars,cardinality,s)
    real_vals[s], estimates[s], variances[s], _, times[s] = scenario(problem_generator, num_problems,
                                           num_repeats, num_samples, num_measures, algorithms, rnd_seed=seed)

    #plot_results(real_vals[s], estimates[s], variances[s], None, num_samples, num_measures, logscale=[(0,1),(1,1)])

######################## WHAT DO WE WANT TO SAVE? ########################
data = {"real_vals": real_vals, "estimates": estimates, "variances": variances, "times": times, 
        "n_rep": num_repeats, "num_samples": num_samples, "num_measures": num_measures, "deltas": deltas,
        "n_vars": n_vars, "cardinality": cardinality, "seed": seed
       }

f_pckl_name = "results_paper/exp200528c_seed_"+str(seed)+"_nvars_"+str(n_vars)+"_"+str(int(time.time()))+".pickle"
with open(f_pckl_name, 'wb') as f_pckl:
    pickle.dump(data, f_pckl, pickle.HIGHEST_PROTOCOL)


In [ ]:
#4x4 grids
files = ["exp200528c_seed_7_delta_2.0_4_2_100_1591338764.pickle",
         "exp200528c_seed_7_delta_3.5_4_2_100_1591339499.pickle",
         "exp200528c_seed_7_delta_5.0_4_2_100_1591339096.pickle",
         "exp200528c_seed_7_delta_10.0_4_2_100_1591339387.pickle",
         "exp200528c_seed_17_delta_2-3.5-5_4_2_1591118341.pickle",
         "exp200528c_seed_17_delta_10_4_2_1591346842.pickle"]


algs =      ["VIS-Rh-1","VIS-Rh-2","VIS-Rh-3","VIS-Rh-4","VIS-Rh-5","VIS-Rh-6","VIS-Rh-7","VIS-Rh-8","VIS-Rh-9",
            "VIS-Rh-10","VIS-Rh-11","VIS-Rh-12","VIS-Rh-13","VIS-Rh-14"]
to_show = range(len(algs))

pnt = 2**7-1

results = {}
for s in [2,3.5,5,10]:
    results[s] = {}
    for a in algs:
        results[s][a] = []

for f in files:
    with open('results_paper/' + f, 'rb') as handle:
        res = pickle.load(handle)

        for s in res["deltas"]:
            for a in algs:
                #results[s][a].append(np.mean(res["variances"][s][a], axis=(0, 1))[pnt])
                results[s][a].append(np.mean(res["times"][s][a]))

for s in [2,3.5,5,10]:
    for a in algs:
        results[s][a] = np.mean(results[s][a])
    results[s] = np.array(list(results[s].values()))[to_show]

results

#results_var = results
results_time = results

In [ ]:
#5x5 grids

files = ["exp200528c_seed_7_delta_2.0_5_2_100_1591949388.pickle",
         "exp200528c_seed_7_delta_3.5_5_2_100_1591938533.pickle",
         "exp200528c_seed_7_delta_5.0_5_2_100_1591943753.pickle",
         "exp200528c_seed_7_delta_10.0_5_2_100_1591955159.pickle",
         "exp200528c_seed_17_delta_2.0_5_2_100_1591930963.pickle",
         "exp200528c_seed_17_delta_3.5_5_2_100_1591943629.pickle",
         "exp200528c_seed_17_delta_5.0_5_2_100_1591949072.pickle",
         "exp200528c_seed_17_delta_10.0_5_2_100_1591950488.pickle"]

algs = ["VIS-Rh-1","VIS-Rh-2","VIS-Rh-3","VIS-Rh-4","VIS-Rh-5","VIS-Rh-6","VIS-Rh-7","VIS-Rh-8","VIS-Rh-9",
        "VIS-Rh-10","VIS-Rh-11","VIS-Rh-12","VIS-Rh-13","VIS-Rh-14","VIS-Rh-15","VIS-Rh-16","VIS-Rh-17",
        "VIS-Rh-18","VIS-Rh-19","VIS-Rh-20","VIS-Rh-21","VIS-Rh-22","VIS-Rh-23","VIS-Rh-24"]
to_show = range(len(algs))#[0,3,6,9,12,14,17,20,23]

pnt = 2**7-1

results = {}
for s in [2,3.5,5,10]:
    results[s] = {}
    for a in algs:
        results[s][a] = []

for f in files:
    with open('results_paper/' + f, 'rb') as handle:
        res = pickle.load(handle)

        for s in res["deltas"]:
            for a in algs:
                #results[s][a].append(np.mean(res["variances"][s][a], axis=(0, 1))[pnt])
                results[s][a].append(np.mean(res["times"][s][a]))

for s in [2,3.5,5,10]:
    for a in algs:
        results[s][a] = np.mean(results[s][a])
    results[s] = np.array(list(results[s].values()))[to_show]

results

#results_var = results
results_time = results

In [ ]:
sel_delta = 10

fig,ax = plt.subplots(figsize=(6,4))

lns1= ax.plot(np.arange(len(algs))+1, results_time[3.5], color="tab:orange", marker="o", fillstyle='none', 
              label="Time, $\\tau=3.5$")
lns2= ax.plot(np.arange(len(algs))+1, results_time[10], color="tab:orange", marker="*", label="Time, $\\tau=10$")

ax.set_xlabel("N. maximum of vars. ($v_{max}$)",fontsize=14)
ax.set_ylabel("Time", fontsize=14)

# twin object for two different y-axis on the sample plot
ax2=ax.twinx()

lns3= ax2.plot(np.arange(len(algs))+1, results_var[3.5],color="tab:blue", linestyle=':', marker="o", 
               fillstyle='none', label="Variance, $\\tau=3.5$")
lns4= ax2.plot(np.arange(len(algs))+1, results_var[10],color="tab:blue", linestyle=':', marker="*", 
               label="Variance, $\\tau=10$")
ax2.set_ylabel("Variance", fontsize=14)
x1, x2, y1, y2 = ax2.axis()
ax2.axis((x1, x2, np.min(results_var[3.5])*0.5, np.max(results_var[10])*1.5))
ax2.set_yscale('log')

plt.tight_layout()

lns = lns1+lns2+lns3+lns4
labs = [l.get_label() for l in lns]
ax.legend(lns, labs, loc=6)

plt.show()


## Experiment: time graphics


In [ ]:
algorithms = [ VIS_R_sp(r=2, step=0.5),
               VIS_R(r=2, step=0.5),
               VIS_I(),
               VIS_Ru(r=2) ]

vars_x_card = [(4,2),(4,3),
               (5,2),(5,3)]

deltas = np.array([2, 3.5, 5, 10])
f_model = lambda n,d,s : GraphModel(ising_grid(n=n,d=d,sigp=s))
                

num_problems = 100

proj_times = {}

for n_vars, cardinality in vars_x_card:
    proj_times[(n_vars, cardinality)] = []
    print ("n_vars, card = ", n_vars, ",", cardinality)

    for d in deltas:
        print ("delta = ", d)
        problem_generator = lambda : f_model(n_vars, cardinality, d)
        proj_times[triplet].append(scenario_time_consumption(problem_generator, num_problems, algorithms))

    plot_times(proj_times[(n_vars, cardinality)], xticks=deltas, logscale=True)

######################## THIS IS WHAT WE SAVE ########################
data = {"proj_times": proj_times, "num_problems": num_problems, "deltas": deltas, "vars_x_card": vars_x_card}

f_pckl_name = "results_paper/exp200528b_nprob_"+str(num_problems)+"_len_vxc_"+str(len(vars_x_card))+"_"+str(int(time.time()))+".pickle"
with open(f_pckl_name, 'wb') as f_pckl:
    pickle.dump(data, f_pckl, pickle.HIGHEST_PROTOCOL)


In [ ]:
'''files = ["exp200528b_delta_2.0_4_2_nprob_20_1591958045.pickle",
         "exp200528b_delta_3.5_4_2_nprob_20_1591958190.pickle",
         "exp200528b_delta_5.0_4_2_nprob_20_1591958220.pickle",
         "exp200528b_delta_10.0_4_2_nprob_20_1591958235.pickle"]
clave= (4,2)

deltas = np.array([2, 3.5, 5, 10])

proj_times_all_delta = []
for f in files:
    with open('results_paper/' + f, 'rb') as handle:
        res = pickle.load(handle)

        for s in res["deltas"]:
            proj_times_all_delta.append(res["proj_times"][clave][0])
my_plot_times(proj_times_all_delta, xticks=deltas, logscale=True)
'''            

## Experiment 200528: some graphics for paper

only a few samples, only basic VIS approaches

In [ ]:
algorithms = [ VIS_R_sp(r=2, step=0.5),
               VIS_R(r=2, step=0.5),
               VIS_I(),
               VIS_Ru(r=2) ]

n_vars = 4
cardinality = 2

deltas = np.array([2, 3.5, 5, 10])
f_model = lambda n,d,s : GraphModel(ising_grid(n=n,d=d,sigp=s))
                

seed = 7

num_problems = 1
num_repeats = 100
num_samples = cardinality**13
num_measures = cardinality**7

estimates = {}
variances = {}
real_vals = {}

for s in deltas:
    print ("Delta = ",s)
    problem_generator = lambda : f_model(n_vars,cardinality,s)
    real_vals[s], estimates[s], variances[s], _,_ = scenario(problem_generator, num_problems,
                                           num_repeats, num_samples, num_measures, algorithms, rnd_seed=seed)

    plot_results(real_vals[s], estimates[s], variances[s], None, num_samples, num_measures, logscale=[(0,1),(1,1)])

######################## THIS IS WHAT WE SAVE ########################
data = {"real_vals": real_vals, "estimates": estimates, "variances": variances, 
        "n_rep": num_repeats, "num_samples": num_samples, "num_measures": num_measures,
        "n_vars": n_vars, "cardinality": cardinality, "seed": seed
       }

f_pckl_name = "results_paper/exp200528_seed_"+str(seed)+"_nrep_"+str(num_repeats)+"_"+str(int(time.time()))+".pickle"
with open(f_pckl_name, 'wb') as f_pckl:
    pickle.dump(data, f_pckl, pickle.HIGHEST_PROTOCOL)


In [ ]:
'''files = ["exp200707_seed_7_delta_2.0_nsamp_8192_nrep_1000_1594126567.pickle",
         "exp200707_seed_7_delta_3.5_nsamp_8192_nrep_1000_1594126421.pickle",
         "exp200707_seed_7_delta_5.0_nsamp_8192_nrep_1000_1594126561.pickle",
         "exp200707_seed_7_delta_10.0_nsamp_8192_nrep_1000_1594126526.pickle",
         "exp200707_seed_17_delta_2.0_nsamp_8192_nrep_1000_1594126192.pickle",
         "exp200707_seed_17_delta_3.5_nsamp_8192_nrep_1000_1594126452.pickle",
         "exp200707_seed_17_delta_5.0_nsamp_8192_nrep_1000_1594126386.pickle",
         "exp200707_seed_17_delta_10.0_nsamp_8192_nrep_1000_1594126197.pickle"]

for f in files:
    with open('results_paper/' + f, 'rb') as handle:
        res = pickle.load(handle)

    for s in res["deltas"]:#np.array([2, 3.5, 5, 10]):
        my_plot_results(res["real_vals"][s],res["estimates"][s], res["variances"][s], None, 
                        res["num_samples"], res["num_measures"],logscale=[1], limits_without_smallest=False,
                        to_show_algs=["VIS-Rh-H","VIS-Rm","VIS-I","VIS-R"], 
                        names_to_show=["VIS-Rh","VIS-Rm","VIS-I","VIS-R"])'''


## Experiment 200429: relative alpha vs fixed alpha

Only mixture proposals are considered

In [ ]:
n_vars = 4
cardinality = 2

deltas = np.array([2,3.5,5,10])# np.linspace(0.,2.,3)
f_model = lambda n,d,s : GraphModel(ising_grid(n=n,d=d,sigp=s))

algorithms = [ VIS_Sp_I_cont(r=2,rate_i=0.5,minoriting_factor=0.5,step=0.5, control_var=True),
               VIS_Sp_I_est(r=2,step=0.5, control_var=True, deterministic_i=True),
               VIS_Ru(r=2)
             ]

rates_i =np.array([0.5,0.25,0.1,0.05])#

seed = 7

num_repeats = np.array([10,100,1000])
num_samples = cardinality**13 #(n_vars**2)
num_measures = 2**7

estimates = {}
variances = {}
theo_variances = {}
real_vals = {}

for n_rep in num_repeats:
    for s in deltas:
        print ("Delta = ",s)
        problem_generator = lambda : f_model(n_vars,cardinality,s)
        real_vals[s], estimates[s], variances[s], theo_variances[s] = scenario_with_many_samplings_and_rates_i(problem_generator, 
                                               n_rep, num_samples, num_measures, algorithms, rates_i, rnd_seed=seed)

        plot_results(real_vals[s],estimates[s], variances[s], theo_variances[s], 
                     num_samples, num_measures)

    ######################## WHAT DO WE WANT TO SAVE? ########################
    data = {"real_vals": real_vals, "estimates": estimates, "variances": variances, "theo_variances": theo_variances,
            "n_rep": n_rep, "num_samples": num_samples, "num_measures": num_measures,
            "n_vars": n_vars, "cardinality": cardinality, "seed": seed
           }

    f_pckl_name = "results/exp200429_seed_"+str(seed)+"_nrep_"+str(n_rep)+"_"+str(int(time.time()))+".pickle"
    with open(f_pckl_name, 'wb') as f_pckl:
        pickle.dump(data, f_pckl, pickle.HIGHEST_PROTOCOL)
    #with open(f_pckl_name, 'rb') as f_pckl:
    #    datab = pickle.load(f_pckl)


In [ ]:
'''with open('results/exp200429_seed_7_nrep_1000_delta_10.0_1588706160.pickle', 'rb') as handle:
    res = pickle.load(handle)

for s in [10]:
    my_plot_results(res["real_vals"][s],res["estimates"][s], res["variances"][s], res["theo_variances"][s], 
                    res["num_samples"], res["num_measures"],logscale=[1], limits_without_smallest=True,
                    to_show_algs=["VIS-R-u","VIS-Sp-I-cont","VIS-Sp-I-est-0.5","VIS-Sp-I-est-0.25",
                                  "VIS-Sp-I-est-0.1","VIS-Sp-I-est-0.05"], 
                    names_to_show=["VIS-R","VIS-Rx-rel","VIS-Rx-0.5","VIS-Rx-0.25","VIS-Rx-0.1","VIS-Rx-0.05"])
'''

## Experiment 200423: adding a deterministic method and simple MC

In [ ]:
n_vars = 4
cardinality = 2

deltas = np.array([2,3.5,5,10])# np.linspace(0.,2.,3)
f_model = lambda n,d,s : GraphModel(ising_grid(n=n,d=d,sigp=s))


algorithms = [ SimpleMonteCarlo(ext_name="-rp", random_proposal=True),
               DeterministicEstimation(ext_name="-rp", random_proposal=True),
               VIS_Sp_I_est(r=2,step=0.5, control_var=True, deterministic_i=True),
               VIS_R_sp(r=2,step=0.5, ext_name="-H"),
               VIS_R(r=2,step=0.5),
               VIS_I(),
               VIS_Ru(r=2)
             ]

rates_i =np.array([0.5,0.25,0.1,0.05])

seed = 17

num_repeats = np.array([10,100,1000])
num_samples = cardinality**(n_vars**2)
num_measures = 2**7

estimates = {}
variances = {}
theo_variances = {}
real_vals = {}

for n_rep in num_repeats:
    for s in deltas:
        print ("Delta = ",s)
        problem_generator = lambda : f_model(n_vars,cardinality,s)
        real_vals[s], estimates[s], variances[s], theo_variances[s] = scenario_with_many_samplings_and_rates_i(problem_generator, 
                                               n_rep, num_samples, num_measures, algorithms, rates_i, rnd_seed=seed)

        plot_results(real_vals[s],estimates[s], variances[s], theo_variances[s], 
                     num_samples, num_measures)

    ######################## WHAT DO WE WANT TO SAVE? ########################
    data = {"real_vals": real_vals, "estimates": estimates, "variances": variances, "theo_variances": theo_variances,
            "n_rep": n_rep, "num_samples": num_samples, "num_measures": num_measures,
            "n_vars": n_vars, "cardinality": cardinality, "seed": seed
           }

    f_pckl_name = "results/exp200408_seed_"+str(seed)+"_nrep_"+str(n_rep)+"_"+str(int(time.time()))+".pickle"
    with open(f_pckl_name, 'wb') as f_pckl:
        pickle.dump(data, f_pckl, pickle.HIGHEST_PROTOCOL)
    #with open(f_pckl_name, 'rb') as f_pckl:
    #    datab = pickle.load(f_pckl)


In [ ]:
'''with open('results/exp200423_seed_17_nrep_1000_delta_10.0_1587872775.pickle', 'rb') as handle:
    res = pickle.load(handle)

for s in res["deltas"]:
    my_plot_results(res["real_vals"][s],res["estimates"][s], res["variances"][s], res["theo_variances"][s], 
                    res["num_samples"], res["num_measures"],logscale=[1], limits_without_smallest=True,
                    to_show_algs=["VIS-R-sp-H","VIS-R","VIS-I","VIS-R-u","DEst-rp","sMC-rp"], 
                    names_to_show=["VIS-Rh","VIS-Rm","VIS-I","VIS-R","DEst","sMC"])'''

## Experiment 200408: a single problem, diff times of repetitions

In this experiment, we want to see how the number of repetitions and samples affect the different approaches.
To do so, a single problem is fixed (by using the seed)

In [ ]:
n_vars = 4
cardinality = 2

deltas = np.array([2,3.5,5,10])# np.linspace(0.,2.,3)
f_model = lambda n,d,s : GraphModel(ising_grid(n=n,d=d,sigp=s))

#deltas = np.array([0, 0.5, 1])# np.linspace(0.,2.,3)
#f_model = lambda n,d,s : GraphModel(my_ising_grid(n=n,d=d,sigp=s,sigu=1))

                
algorithms = [ VIS_Sp_I_est(r=2,step=0.5,fixed_i=False, control_var=True, deterministic_i=True),
               VIS_R_sp(r=2,step=0.5, ext_name="-H"),
               VIS_R(r=2,step=0.5),
               #VIS_A(alpha=2,step=0.5),
               VIS_I(),
               VIS_Ru(r=2)
             ]

rates_i =np.array([0.5,0.25,0.1,0.05])

seed = 17

num_repeats = np.array([10,100,1000])
num_samples = 5000
num_measures = 100
                          
estimates = {}
variances = {}
theo_variances = {}
real_vals = {}

for n_rep in num_repeats:
    for s in deltas: #tqdm(weights, desc='Weight'):
        print ("Delta = ",s)
        problem_generator = lambda : f_model(n_vars,cardinality,s)
        real_vals[s], estimates[s], variances[s], theo_variances[s] = scenario_with_many_samplings_and_rates_i(problem_generator, 
                                               n_rep, num_samples, num_measures, algorithms, rates_i, rnd_seed=seed)

        plot_results(real_vals[s],estimates[s], variances[s], theo_variances[s], 
                     num_samples, num_measures)

    ######################## WHAT DO WE WANT TO SAVE? ########################
    data = {"real_vals": real_vals, "estimates": estimates, "variances": variances, "theo_variances": theo_variances,
            "n_rep": n_rep, "num_samples": num_samples, "num_measures": num_measures,
            "n_vars": n_vars, "cardinality": cardinality, "seed": seed
           }

    f_pckl_name = "results/exp200408_seed_"+str(seed)+"_nrep_"+str(n_rep)+"_"+str(int(time.time()))+".pickle"
    with open(f_pckl_name, 'wb') as f_pckl:
        pickle.dump(data, f_pckl, pickle.HIGHEST_PROTOCOL)
    #with open(f_pckl_name, 'rb') as f_pckl:
    #    datab = pickle.load(f_pckl)


## Experiment 200403: different options for mixture proposal

Deterministic or not: sample N from q_mix or N_i from each q_i

Fixed_i or not: return always max-prob sample according to i-projection (true) or sample from i-projection (false)

*** there were still not control variates

In [ ]:
np.random.seed(7)
n_vars = 4
cardinality = 2

deltas = np.array([2,3.5,5,10])# np.linspace(0.,2.,3)
f_model = lambda n,d,s : GraphModel(ising_grid(n=n,d=d,sigp=s))

                
algorithms = [ VIS_Sp_I(r=2,step=0.5,rate_i=0.5,fixed_i=False, deterministic_i=True, ext_name="-0.5"),
               VIS_Sp_I(r=2,step=0.5,rate_i=0.25,fixed_i=False, deterministic_i=True, ext_name="-0.25"),
               VIS_Sp_I(r=2,step=0.5,rate_i=0.1,fixed_i=False, deterministic_i=True, ext_name="-0.1"),
               VIS_Sp_I(r=2,step=0.5,rate_i=0.05,fixed_i=False, deterministic_i=True, ext_name="-0.05"),
               VIS_R_sp(r=2,step=0.5, ext_name="-H"),
               VIS_R(r=2,step=0.5),
               VIS_I(),
               VIS_Ru(r=2),
             ]

num_problems = 10
num_repeats = 3
num_samples = 5000
num_measures = 100
                          
estimates = {}
variances = {}
theo_variances = {}
real_vals = {}


for s in deltas: #tqdm(weights, desc='Weight'):
    print ("Delta = ",s)
    problem_generator = lambda : f_model(n_vars,cardinality,s)
    real_vals[s], estimates[s], variances[s], theo_variances[s] = scenario(problem_generator, num_problems, 
                                                                           num_repeats, num_samples, 
                                                                           num_measures, algorithms)
    #print(estimates[s])
    plot_results(real_vals[s],estimates[s], variances[s], theo_variances[s], 
                 num_samples, num_measures)
                          